# HEATMAP

In [2]:
%load_ext autoreload
%autoreload 2

from bokeh.io import output_notebook, show
output_notebook()
from bokeh.models import HoverTool, ColorBar
from bokeh.plotting import figure, gridplot
from bokeh.transform import linear_cmap
import bokeh.palettes as bpl
from bokeh.models.sources import ColumnDataSource
import numpy as np
import pandas as pd
from bokeh.models import FixedTicker

def heatmap_plot(df, Xcol, Ycol, Zcol, hower_cols=[], name='', DAYS_NORM=7, w=1500, h=400):
  data = df.copy()
  data[[Xcol, Ycol]] = data[[Xcol, Ycol]].astype('str')
  source = ColumnDataSource(data=data)
  tools = ['save', 'box_select', 'reset', HoverTool(tooltips=[(i, '@'+i) for i in [Xcol, Ycol] + hower_cols])]
  fig1 = figure(title=name or Zcol, x_range=data[Xcol].unique(), y_range=data[Ycol].unique(), tools=tools, width=w, height=h)
  fig1.grid.visible = False
  ncolors = min(sum(data[Zcol]>=0), 256)
  mapper = linear_cmap(field_name=Zcol, palette=bpl.cividis(ncolors), low=0, high=data[Zcol].max(), low_color='#000000')
  fig1.rect(Xcol, Ycol, width=1, height=1, fill_color=mapper, line_color=None, source=source)
  fig1.xaxis.axis_label = Xcol
  fig1.yaxis.axis_label = Ycol
  colorbar = ColorBar(color_mapper=mapper['transform'], label_standoff=12, location=(0, 0))
  fig1.add_layout(colorbar, 'right')
  fig1.xaxis.major_label_orientation = np.pi/4  

  col = 'days_per_close'
  fig2 = figure(title=col.replace('_', ' '), x_range=data[Xcol].unique(), y_range=data[Ycol].unique(), tools=tools, width=w, height=h)

  fig2.grid.visible = False
  ncolors = min(data[col].nunique(), 256)
  mapper = linear_cmap(field_name=col, palette=bpl.cividis(ncolors)[::-1], low=data[col].min(), high=DAYS_NORM*1.5, high_color='#000000')
  fig2.rect(Xcol, Ycol, width=1, height=1, fill_color=mapper, line_color=None, source=source)
  fig2.xaxis.axis_label = Xcol
  fig2.yaxis.axis_label = Ycol
  colorbar = ColorBar(color_mapper=mapper['transform'], label_standoff=12, location=(0, 0))
  fig2.add_layout(colorbar, 'right')
  fig2.xaxis.major_label_orientation = np.pi/4

  grid = gridplot([[fig1], [fig2]])
  show(grid)

def res_to_metrics(df_res, DAYS_NORM): 
  means = (df_res.mean(axis=0)/DAYS_NORM).copy()
  n_close = means['n_stops']+means['n_exits']
  means['days_per_close'] = 1/n_close if n_close else 0
  means['pnl_per_day'] = means['pnl']/means['days_per_close'] if means['days_per_close'] else -1
  return means

def plot_histmap(res, fix_params, X='window_MA', Y='c_enter', Z='pnl', DAYS_NORM=7, w=1500,h=400, name=None):
  hover_columns = ['pnl','pnl_per_day','days_per_close','pnl_positive','n_positive','n_stops','pnl_stops','pnl_exits','n_exits']

  data = pd.DataFrame(columns = [X, Y]+hover_columns, index = range(len(res)))
  c = 0
  for df_res, params in res:
    for k in fix_params:
      if params[k] not in fix_params[k]:
        break
    else:
      if ((data[X]==params[X]) & (data[Y]==params[Y])).sum():
        idx = data[(data[X]==params[X]) & (data[Y]==params[Y])].index[0]
        data.loc[idx, hover_columns] += res_to_metrics(df_res, DAYS_NORM)
      else:
        data.loc[c, [X,Y]] = params[X], params[Y]
        data.loc[c, hover_columns] = res_to_metrics(df_res, DAYS_NORM)
        c += 1

  data = data.dropna()

  heatmap_plot(data, X, Y, Z, hower_cols=hover_columns, 
              name=f"c_stop={fix_params['c_stop'][0]}" if name is None else name, w=w, h=h)
  return data

Loading BokehJS ...

In [37]:
import numpy as np
import pandas as pd
import pickle

X = 'window_MA'
Y = 'c_enter'
path = '/home/philurame/SPVCapital/cointegrate/framework/data/grid/res_grid.pkl'

with open(path, 'rb') as f: res = pickle.load(f)
res = [(df_res, {'window_MA':params['window_MA'], 'c_enter':params['c_enter'], 'c_stop':params['c_stop']}) for df_res, params in res 
       if params['coeffs']['type']=='mean']

In [72]:
grid_params = dict(
  window_MA = np.arange(2000,31000,500),
  c_enter   = np.arange(1, 7, 0.5),
  c_stop    = np.arange(4, 10, 0.5),
)

fix_params = dict(
  c_stop = [9.5],
)

data = plot_histmap(res, fix_params, X, Y, 'pnl', w=1500, h=400)


fix_params = dict(
  c_stop = [7],
)

data = plot_histmap(res, fix_params, X, Y, 'pnl', w=1500, h=400)

# ?

In [1]:
import pickle
res = pickle.load(open('/home/philurame/SPVCapital/cointegrate/framework/data/grid/res_BTCETH.pkl', 'rb'))

In [3]:
lags = list(set([i[0] for i in res]))

In [4]:
X = 'window_MA'
Y = 'c_enter'
lag = 4
print(lags[lag])
res_lag = [(i[1], i[2]) for i in res if i[0]==lags[lag]]

grid_params = dict(
  window_MA = np.arange(1000,30000,1000),
  c_enter   = np.arange(1, 7, 1),
)
fix_params = dict(c_stop=[9.5])

data = plot_histmap(res_lag, fix_params, X, Y, 'pnl', w=800, h=200, name=f'lag={lags[lag]}')

# data = plot_histmap(res, fix_params, X, Y, 'pnl', w=1500, h=400)

1000SHIBUSDT
